If running out of disk memory run <br>
`cd /tmp/`<br>
`rm -r *`

In [7]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
import tensorflow_datasets as tfds
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import run_classifier_with_tfhub
#https://github.com/google-research/bert.git
import sys
from tensorflow import keras
import os
import re
from transformers import *
import numpy as np
from tensorflow.python.lib.io import file_io
import pickle
import gc
import threading
import logging

Neither PyTorch nor TensorFlow >= 2.0 have been found.Models won't be available and only tokenizers, configurationand file/data utilities can be used.


# Initialize TPU

In [11]:
# Based on -> https://towardsdatascience.com/https-medium-com-chaturangarajapakshe-text-classification-with-transformer-models-d370944b50ca
# Active TPU'1
TPU_ADDRESS = "node-2"
TPU_ZONE = "us-central1-f"
USE_TPU =True
NUM_TPU_CORES = 8
#len(tf.config.experimental.list_logical_devices('TPU'))

#tf.config.experimental_connect_to_cluster(resolver)
#tf.tpu.experimental.initialize_tpu_system(resolver)
#tpu_strategy = tf.distribute.experimental.TPUStrategy(resolver)
#print("All devices: ", tf.config.experimental.list_logical_devices('TPU'))

# Setup TPU related config
#tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
#NUM_TPU_CORES = 8

#https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb#scrollTo=191zq3ZErihP
#with tf.Session(TPU_ADDRESS) as session:
   # print('TPU devices:')
   # pprint.pprint(session.list_devices())    
    #contrib.cloud.configure_gcs(session)
    

All devices:  8


# Setting output directory

In [12]:
def getDir(bucket, output_dir):
    return 'gs://{}/{}'.format(bucket, output_dir)

In [13]:
OUTPUT_DIR = "bertResults"
DO_DELETE = True
USE_BUCKET =True
BUCKET = "patents-research"

if USE_BUCKET:
    OUTPUT_DIR = getDir(BUCKET, OUTPUT_DIR)


if DO_DELETE:
    try:
        tf.gfile.DeleteRecursively(OUTPUT_DIR)
    except:
        # doesn't matter if the directory didn't exist
        pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: gs://patents-research/bertResults *****


# Load Data Set

We classify high impact patents following (Ahuja and Lampert, 2001) as the patents
 within a given cohort receiving the most citations by other patents within the following 5 year window. Thereafter, for every year we sorted
the patents applied for in that year on the basis
of their citation weights and identified the top 1
percent of patents for that year as breakthrough
inventions. This procedure ensures that each patent is compared in its importance only to other
patents of the same yea
https://towardsdatascience.com/https-medium-com-chaturangarajapakshe-text-classification-with-transformer-models-d370944b50ca

In [15]:
def loadPdData(gsPath):
    return pd.read_csv(gsPath, sep = "\t")

def generateLable(dataset):
    # convert to datetime
    dataset['publication_date'] = pd.to_datetime(dataset['publication_date'], errors="coerce",format="%Y-%m-%d")
    dataset = dataset.sort_values('publication_date', ascending = False)

    #drop if date is NaN - only one 1082-03-15
    dataset = dataset[dataset.publication_date.isnull() == False]
    
    # calculate the top 1% by publication date - give it label 1
    top1_perc =  dataset.groupby(dataset.publication_date.dt.year)["fwrdCitations_5"].transform(lambda x: x.quantile(.99))
    dataset["label"] = dataset["fwrdCitations_5"] >= top1_perc
    
    # calculate top 5% by publication date - give it label 2
    top5_perc = dataset.groupby(dataset.publication_date.dt.year)["fwrdCitations_5"].transform(lambda x: x.quantile(.95))
    dataset["label"] = np.where(np.logical_and(dataset["fwrdCitations_5"] >= top5_perc, dataset["label"]==0), 2, dataset["label"])
    
    return dataset

def saveToGcloud(path,data,isPandas = False ):
    '''Saves to gcloud so we dont have to do this long ass step every time'''
    if isPandas:
        data.to_csv(path, index=False, sep="\t")
    else:
        with file_io.FileIO(path, mode='w') as f:
            pickle.dump(data,f)


def readFromGcloud(path, isPandas = False):
    if isPandas:
        return pd.read_csv(path,sep="\t" )
    else:
        with file_io.FileIO(path, mode='rb') as f:
            return pickle.load(f)
        
def loadData(loadFromCloud=False):
    print("Max seq length {}".format(MAX_SEQ_LENGTH))
    if loadFromCloud:
        print(f'Loading saved data from cloud!')
        train_features = readFromGcloud(TRAIN_TFRecord_PATH)
        test_features =readFromGcloud(TEST_TFRecord_PATH)
        print(f'Finished loading saved data from cloud!')

    else:
        print(f'Loading data!')
        dataset = loadPdData(DATA_PATH)
        print(f'Finised loading data!')
        dataset = generateLable(dataset)
        print(f'Test/Train Split!')
        train,test=train_test_split(dataset, test_size=0.2)
        del dataset
        gc.collect()
        print(f'Finished Test/Train Split!')

        print('Saving Test/Train Split to gCloud')
        #saveToGloud(TRAIN_DF_PATH,train,isPandas=True)
        #saveToGloud(TEST_DF_PATH,test,isPandas=True)
        print('Finished Saving Test/Train Split to gCloud!')

        print("Use the InputExample class from BERT's run_classifier code to create examples from the data")
        train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                           text_a = x[DATA_COLUMN], 
                                                                           text_b = None, 
                                                                           label = x[LABEL_COLUMN]), axis = 1)
        del train
        gc.collect()
        test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                           text_a = x[DATA_COLUMN], 
                                                                           text_b = None, 
                                                                           label = x[LABEL_COLUMN]), axis = 1)
        del test
        gc.collect()
        
        print("Finished using  InputExample class from BERT's run_classifier code to create examples from the data")

        print("Convert our train and test features to InputFeatures that BERT understands")
        tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased') # get scientific tokenizer + pointer to the model

        train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
        saveToGcloud(TRAIN_TFRecord_PATH,train_features)
        del train_InputExamples
        gc.collect()
        
        test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
        saveToGcloud(TEST_TFRecord_PATH,test_features) 
        del test_InputExamples
        gc.collect()
        print("Finished converting  train and test features to InputFeatures that BERT understands")
    return train_features, test_features


In [16]:
# The path to the data
DATA_PATH = "gs://patents-research/patent_research/data_frwdcorrect.tsv"
TRAIN_DF_PATH= "gs://patents-research/patent_research/{}".format("bert_train_df.tsv") 
TEST_DF_PATH="gs://patents-research/patent_research/{}".format("bert_test_df.tsv")
DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1, 2] 



# Data Preprocessing + Loading Training 
**(We load Test asyncronously later)**

Converting data into format that bert understands. We use https://github.com/allenai/scibert instead of the standard tokenizer/trained model ref= https://www.aclweb.org/anthology/D19-1371.pdf

In [17]:
#HUB_MODULE = "https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/tensorflow_models/scibert_scivocab_uncased.tar.gz"
TRAIN_TFRecord_PATH= "gs://patents-research/patent_research/{}".format("train_features_512.pickle")
TEST_TFRecord_PATH= "gs://patents-research/patent_research/{}".format("test_features_512.pickle")
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 512 #default sequence is 128



#### Download train data

In [18]:

train_features = readFromGcloud(TRAIN_TFRecord_PATH)

Loading saved data from cloud!
Finished loading saved data from cloud!


#### Download test data 

Threading this so we can train immediately, and we are not stuck waiting on test data to download when we don't need it immediately.

In [ ]:
def worker_downloadTestData(name):
    """
    Worker so we can download test data asynch
    """
    logging.info("Thread %s: starting", name)
    global test_features
    train_features = readFromGcloud(TEST_TFRecord_PATH)
    logging.info("Thread %s: finishing", name)   
    
# Set up threading 
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,datefmt="%H:%M:%S")
getTestData_thread = threading.Thread(target=downloadTest, args=(1,))
getTestData_thread.start() #async download of test data 

# Creating a model

In [19]:
def create_model(is_training, input_ids, input_mask, segment_ids, labels,
                 num_labels, bert_hub_module_handle, dropout):
    """Creates a classification model."""
    tags = set()
    if is_training:
        tags.add("train")
    bert_module = hub.Module(bert_hub_module_handle, tags=tags, trainable=True)
    bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
    bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)


    output_layer = bert_outputs["pooled_output"]

    hidden_size = output_layer.shape[-1].value

    output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):
        if is_training:
            # I.e., 0.1 dropout
            output_layer = tf.nn.dropout(output_layer, keep_prob=dropout)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        probabilities = tf.nn.softmax(logits, axis=-1)
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)

        return (loss, per_example_loss, logits, probabilities)

In [24]:
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps, use_tpu, bert_hub_module_handle, dropout):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):  
        """The `model_fn` for TPUEstimator."""

        tf.logging.info("*** Features ***")
        for name in sorted(features.keys()):
            tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_training = (mode == tf.estimator.ModeKeys.TRAIN)

        (total_loss, per_example_loss, logits, probabilities) = create_model(
                is_training, input_ids, input_mask, segment_ids, label_ids, num_labels,
                bert_hub_module_handle, dropout)

        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:
            train_op = optimization.create_optimizer(
                    total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
                  mode=mode,
                  loss=total_loss,
                  train_op=train_op)
        elif mode == tf.estimator.ModeKeys.EVAL:

            def metric_fn(per_example_loss, label_ids, logits):
                predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
                accuracy = tf.metrics.accuracy(label_ids, predictions)
                loss = tf.metrics.mean(per_example_loss)
                true_pos = tf.metrics.true_positives(
                            label_ids,
                            predictions)
                true_neg = tf.metrics.true_negatives(
                            label_ids,
                            predictions)   
                false_pos = tf.metrics.false_positives(
                            label_ids,
                            predictions)  
                false_neg = tf.metrics.false_negatives(
                            label_ids,
                            predictions) 
                return {
                    "true_positives": true_pos,
                    "true_negatives": true_neg,
                    "false_positives": false_pos,
                    "false_negatives": false_neg,
                    "eval_accuracy": accuracy,
                    "eval_loss": loss,
                }

            eval_metrics = (metric_fn, [per_example_loss, label_ids, logits])
            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
              mode=mode,
              loss=total_loss,
              eval_metrics=eval_metrics)
        
        elif mode == tf.estimator.ModeKeys.PREDICT:
            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
              mode=mode, predictions={"probabilities": probabilities})
        else:
            raise ValueError(
              "Only TRAIN, EVAL and PREDICT modes are supported: %s" % (mode))

        return output_spec

    return model_fn

# MODEL PARAMETERS

Example params
- https://github.com/google-research/bert/issues/649 
- https://github.com/google-research/bert/issues/425

In [25]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 256
EVAL_BATCH_SIZE = NUM_TPU_CORES
PREDICT_BATCH_SIZE = NUM_TPU_CORES
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 40.0
DROPOUT_KEEP_PROB = .7
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 100


In [26]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS) 
#int((800000) / 256 * 40) == about 32k
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

## Set up TPU 
https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb#scrollTo=pYVYULZiKvUi

In [22]:
# Setupt TPU related config
tpu_cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_ADDRESS, zone=TPU_ZONE)
tf.tpu.experimental.initialize_tpu_system(tpu_cluster_resolver)
tf.distribute.experimental.TPUStrategy(tpu_cluster_resolver)

ITERATIONS_PER_LOOP = 1000

# Force TF Hub writes to the GS bucket we provide.
os.environ['TFHUB_CACHE_DIR'] =  os.path.join(OUTPUT_DIR,"tfhub_cache")
tf.gfile.MakeDirs(os.path.join(OUTPUT_DIR,"tfhub_cache"))




INFO:tensorflow:Initializing the TPU system: node-2
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Querying Tensorflow master (grpc://10.157.197.34:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 3455271994140814130)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 10245224255260713639)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 16321600984248760639)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 17663426012522639575)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:3,

# Functions to Train + Evaluate Model

In [28]:
def get_run_config(output_dir):
    """
    Used for run configuration when TPU used
    """
    return tf.contrib.tpu.RunConfig(
        cluster=tpu_cluster_resolver,
        model_dir=output_dir,
        save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
        tpu_config=tf.contrib.tpu.TPUConfig(
            iterations_per_loop=ITERATIONS_PER_LOOP,
            num_shards=NUM_TPU_CORES,
            per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

def getEstimator(mode_fn):
    """
    Returns the estimator used to train/eval model
    """
    return tf.estimator.tpu.TPUEstimator(
          use_tpu=True,
          model_fn=mode_fn,
          config=get_run_config(OUTPUT_DIR),
          train_batch_size=BATCH_SIZE,
          eval_batch_size=EVAL_BATCH_SIZE,
          predict_batch_size=PREDICT_BATCH_SIZE,
          eval_on_tpu = True
        ) 

def model_train(estimator):
    """
    Trains the model, rt only good for TPU
    """
    #Set drop_remainder =True to fix a TPU error
    #https://stackoverflow.com/questions/58029896/bert-fine-tuning-with-estimators-on-tpus-on-colab-typeerror-unsupported-operand

    print('***** Started training at {} *****'.format(datetime.now()))
    print('  Num examples = {}'.format(len(train_features)))
    print('  Batch size = {}'.format(BATCH_SIZE))
    tf.logging.info("  Num steps = %d", num_train_steps)
    
    current_time = datetime.now()
    train_input_fn = run_classifier.input_fn_builder(
        features=train_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=True,
        drop_remainder=True)
    estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
    print("Finished: Training took time ", datetime.now() - current_time)

def model_evaluate(estimator):
    """
    Evaluates the model
    """
    print('***** Started evaluation at {} *****'.format(datetime.now()))
    print('  Num examples = {}'.format(len(test_features)))
    print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

    # Eval will be slightly WRONG on the TPU because it will truncate
    # the last batch.
    eval_steps = int(len(test_features) / EVAL_BATCH_SIZE)
    
    eval_input_fn = run_classifier.input_fn_builder(
        features=test_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=True)
    
    result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
    print('***** Finished evaluation at {} *****'.format(datetime.now()))
    output_eval_file = os.path.join(OUTPUT_DIR, "eval","eval_results.txt")
    tf.gfile.MakeDirs(os.path.join(OUTPUT_DIR, "eval"))
    with tf.gfile.GFile(output_eval_file, "w") as writer:
        print("***** Eval results *****")
        for key in sorted(result.keys()):
            print('  {} = {}'.format(key, str(result[key])))
            writer.write("%s = %s\n" % (key, str(result[key])))
      

# Train + Eval model
If evaluate breaks down must rm the lock file in `TFHUB_CACHE_DIR` unless stuck waiting -> https://github.com/tensorflow/hub/issues/579

In [ ]:
mode_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  dropout = DROPOUT_KEEP_PROB,
  use_tpu = USE_TPU,
  bert_hub_module_handle = BERT_MODEL_HUB
)

In [ ]:
estimator = getEstimator(mode_fn) 
model_train(estimator)
getTestData_thread.join() #wait async download of test data to finish - test data used in `model_evaluate`
model_evaluate(estimator)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Using config: {'_model_dir': 'gs://patents-research/bertResults', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.157.197.34:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_work